In [1]:
# clean file testdata de phan tich du lieu
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import datetime as dt


In [2]:

file_path = r'C:\Users\Admin\Desktop\E_commerce_proj\data\datainput\Amazon_Sale_Report_Enhanced.csv' 
raw_df = pd.read_csv(file_path, encoding='latin1')
raw_df.head(5)
raw_df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               15000 non-null  int64  
 1   Order ID            15000 non-null  object 
 2   Date                15000 non-null  object 
 3   Status              15000 non-null  object 
 4   Fulfilment          15000 non-null  object 
 5   Sales Channel       15000 non-null  object 
 6   ship-service-level  15000 non-null  object 
 7   Style               15000 non-null  object 
 8   SKU                 6120 non-null   object 
 9   Category            6030 non-null   object 
 10  Size                15000 non-null  object 
 11  ASIN                15000 non-null  object 
 12  Courier Status      5694 non-null   object 
 13  Qty                 15000 non-null  int64  
 14  currency            5672 non-null   object 
 15  Amount              5685 non-null   float64
 16  ship

Tất cả các giá trị lỗi dạng "####", "###@", "NaN" chuỗi, v.v. đã được chuyển thành NaN thực sự (giá trị thiếu) trong DataFrame

In [3]:
import pandas as pd
import numpy as np

# Danh sách các giá trị lỗi thường gặp cần chuẩn hóa
weird_values = ['####', '###', '##', '#', '###@', 'NaN', 'NULL', '', '##9&', '#8#$', '#2##']

# Thay thế tất cả các giá trị lỗi này trong toàn bộ DataFrame bằng np.nan
df_cleaned = raw_df.replace(to_replace=weird_values, value=np.nan)


In [10]:
# Đếm số lượng giá trị thiếu sau chuẩn hóa
nan_summary = df_cleaned.isnull().sum()
nan_summary = nan_summary[nan_summary > 0].reset_index()
nan_summary.columns = ['Cột', 'Số lượng thiếu (sau chuẩn hóa)']

# Lọc ra các cột có nhiều lỗi nhất (VD: > 5000)
nan_summary = nan_summary[nan_summary['Số lượng thiếu (sau chuẩn hóa)'] > 5000]

# Sắp xếp giảm dần
nan_summary = nan_summary.sort_values(by='Số lượng thiếu (sau chuẩn hóa)', ascending=False)

# Hiển thị kết quả
print("Số lượng giá trị thiếu sau khi chuẩn hóa các lỗi như #@$%")
print(nan_summary)


Số lượng giá trị thiếu sau khi chuẩn hóa các lỗi như #@$%
               Cột  Số lượng thiếu (sau chuẩn hóa)
10    fulfilled-by                           12772
9    promotion-ids                           11125
3         currency                            9328
4           Amount                            9315
2   Courier Status                            9306
6       ship-state                            9090
8     ship-country                            9005
1         Category                            8970
5        ship-city                            8945
0              SKU                            8880


NHẬN XÉT 1

- Một số cột có hơn 50% dữ liệu bị thiếu, nên trong bước tiếp theo chúng ta cần quyết định:

Có nên loại bỏ các dòng này?

Hay sẽ giữ lại và điền dữ liệu thay thế nếu hợp lý?

-Chú ý code sẽ không chạy được nếu file đang được mở

In [20]:
# Gán giá trị mặc định cho cột phụ
df_cleaned['Category'].fillna('Misc', inplace=True)
df_cleaned['Courier Status'].fillna('Unknown', inplace=True)
df_cleaned['ship-city'].fillna('Unknown', inplace=True)
df_cleaned['ship-country'].fillna('Unknown', inplace=True)
df_cleaned['ship-state'].fillna('Unknown', inplace=True)
df_cleaned['SKU'].fillna('Unknown', inplace=True)
df_cleaned['currency'].fillna('INR', inplace=True)
df_cleaned['fulfilled-by'].fillna('Merchant', inplace=True)


# Với Amount là số → dùng median để fill
df_cleaned['Amount'].fillna(df_cleaned['Amount'].median(), inplace=True)

# Bỏ những cột không cần thiết nếu bạn không phân tích
df_cleaned.drop(columns=['promotion-ids', 'ship-postal-code'], inplace=True,errors='ignore')

df_cleaned.to_csv(r'C:\Users\Admin\Desktop\E_commerce_proj\data\datainput\Amazon_Sale_Report_Enhanced_cleaned.csv', index=False, encoding='utf-8-sig')

C:\Users\Admin\AppData\Local\Temp\ipykernel_5044\2969905630.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned['Category'].fillna('Misc', inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_5044\2969905630.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For ex

- Sau khi kiểm tra còn các cột chưa được làm sạch như: SKU, Category... thay thế bằng unknow
- Các hàng nào lỗi quá nhiều được xóa bỏ

